In [1]:
import os
import glob
import shutil
import pathlib
#import cv2
import librosa
import librosa.display
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils import class_weight
from collections import Counter
from scipy.io import wavfile
import tensorflow as tf
import tensorflow_io as tfio
from IPython.display import Audio
#tensorflow
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from tensorflow.keras.layers import GlobalMaxPooling2D, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical


seed_number = 24
tf.random.set_seed(seed_number)
np.random.seed(seed_number)

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
root = "d:/datasets/sound/"
input_dir = os.path.join(root,"data/")
train_dir = os.path.join(input_dir, 'Training_Data/')
val_dir = os.path.join(input_dir, 'val/')
test_dir = os.path.join(input_dir, 'Testing_Data/')
train_img_dir = os.path.join(root, 'img_data/')
val_img_dir = os.path.join(root, 'img_data_val/')

In [ ]:
dataset_dir = [dir for dir in sorted(os.listdir(input_dir)) if os.path.isdir(os.path.join(input_dir, dir))]
label_name = [subdir for subdir in sorted(os.listdir(train_dir)) if os.path.isdir(os.path.join(train_dir, subdir))]

# информация о папках
print(f"Main directories\t: {os.listdir(root)}")
print(f"Dataset sub-directories\t: {dataset_dir}")
print(f"Train set directory\t: {label_name}")

In [3]:
#CONST
SAMPLE_RATE = 16000 # Sampling rate
#duration = 5
#hop_length = 512
FMIN = 20
FMAX = SAMPLE_RATE // 2
#n_mels = 128
N_FFT = 512
Fs = 16000 
#samples =SAMPLE_RATE * duration

In [8]:
#Цикл, который перебирает wav файлы и сохраняет их как спектограммы 
def wav_to_mel(name_classes):
    classes = name_classes.split()
    for g in classes:
        pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
        for filename in os.listdir(input_dir + f'Training_Data/{g}'):
            wav_file_name = input_dir + f'Training_Data/{g}/{filename}'
            y, sr = librosa.load(wav_file_name, mono=True, duration=5)
            print(y.shape)
            plt.specgram(y, NFFT=N_FFT, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
            plt.axis('off');
            plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
            plt.clf()
    return(print('Done'))
            
            

In [10]:
wav_to_mel('human spoof')

(306, 128)
(192, 128)
(312, 128)
(177, 128)
(218, 128)
(492, 128)
(171, 128)
(190, 128)
(188, 128)
(203, 128)
(257, 128)
(355, 128)
(274, 128)
(203, 128)
(188, 128)
(167, 128)
(218, 128)
(312, 128)
(248, 128)
(209, 128)
(242, 128)
(319, 128)
(235, 128)
(600, 128)
(188, 128)
(207, 128)
(224, 128)
(229, 128)
(162, 128)
(175, 128)
(171, 128)
(212, 128)
(195, 128)
(259, 128)
(229, 128)
(165, 128)
(188, 128)
(194, 128)
(188, 128)
(194, 128)
(293, 128)
(604, 128)
(278, 128)
(179, 128)
(250, 128)
(184, 128)
(239, 128)
(188, 128)
(179, 128)
(214, 128)
(612, 128)
(188, 128)
(188, 128)
(276, 128)
(287, 128)
(319, 128)
(272, 128)
(190, 128)
(420, 128)
(188, 128)
(188, 128)
(608, 128)
(597, 128)
(188, 128)
(250, 128)
(330, 128)
(300, 128)
(205, 128)
(210, 128)
(375, 128)
(220, 128)
(188, 128)
(169, 128)
(192, 128)
(175, 128)
(336, 128)
(224, 128)
(619, 128)
(415, 128)
(600, 128)
(164, 128)
(192, 128)
(285, 128)
(422, 128)
(197, 128)
(167, 128)
(190, 128)
(188, 128)
(259, 128)
(507, 128)
(589, 128)

(160, 128)
(323, 128)
(188, 128)
(604, 128)
(175, 128)
(203, 128)
(175, 128)
(188, 128)
(231, 128)
(297, 128)
(327, 128)
(160, 128)
(177, 128)
(180, 128)
(188, 128)
(302, 128)
(302, 128)
(494, 128)
(372, 128)
(623, 128)
(308, 128)
(205, 128)
(164, 128)
(319, 128)
(501, 128)
(188, 128)
(188, 128)
(165, 128)
(194, 128)
(272, 128)
(222, 128)
(165, 128)
(362, 128)
(231, 128)
(300, 128)
(250, 128)
(291, 128)
(608, 128)
(332, 128)
(295, 128)
(188, 128)
(188, 128)
(300, 128)
(212, 128)
(188, 128)
(214, 128)
(627, 128)
(194, 128)
(218, 128)
(197, 128)
(265, 128)
(237, 128)
(188, 128)
(188, 128)
(422, 128)
(314, 128)
(188, 128)
(237, 128)
(171, 128)
(190, 128)
(180, 128)
(184, 128)
(269, 128)
(188, 128)
(284, 128)
(173, 128)
(177, 128)
(212, 128)
(169, 128)
(188, 128)
(169, 128)
(619, 128)
(218, 128)
(188, 128)
(465, 128)
(182, 128)
(282, 128)
(557, 128)
(355, 128)
(589, 128)
(597, 128)
(582, 128)
(186, 128)
(171, 128)
(527, 128)
(197, 128)
(270, 128)
(162, 128)
(250, 128)
(173, 128)
(205, 128)

(201, 128)
(188, 128)
(186, 128)
(188, 128)
(250, 128)
(177, 128)
(192, 128)
(317, 128)
(188, 128)
(162, 128)
(180, 128)
(306, 128)
(188, 128)
(597, 128)
(184, 128)
(284, 128)
(184, 128)
(169, 128)
(169, 128)
(188, 128)
(329, 128)
(218, 128)
(199, 128)
(355, 128)
(188, 128)
(284, 128)
(167, 128)
(171, 128)
(180, 128)
(218, 128)
(218, 128)
(180, 128)
(306, 128)
(222, 128)
(360, 128)
(634, 128)
(190, 128)
(165, 128)
(162, 128)
(160, 128)
(188, 128)
(619, 128)
(274, 128)
(175, 128)
(218, 128)
(349, 128)
(182, 128)
(186, 128)
(190, 128)
(602, 128)
(394, 128)
(531, 128)
(162, 128)
(304, 128)
(162, 128)
(225, 128)
(188, 128)
(188, 128)
(355, 128)
(272, 128)
(634, 128)
(240, 128)
(188, 128)
(612, 128)
(265, 128)
(195, 128)
(452, 128)
(194, 128)
(314, 128)
(205, 128)
(600, 128)
(207, 128)
(162, 128)
(173, 128)
(165, 128)
(179, 128)
(454, 128)
(304, 128)
(299, 128)
(165, 128)
(188, 128)
(589, 128)
(175, 128)
(212, 128)
(329, 128)
(169, 128)
(165, 128)
(175, 128)
(180, 128)
(194, 128)
(188, 128)

(164, 128)
(312, 128)
(355, 128)
(475, 128)
(227, 128)
(222, 128)
(276, 128)
(188, 128)
(634, 128)
(261, 128)
(188, 128)
(239, 128)
(212, 128)
(160, 128)
(169, 128)
(209, 128)
(175, 128)
(300, 128)
(284, 128)
(597, 128)
(237, 128)
(210, 128)
(469, 128)
(634, 128)
(323, 128)
(165, 128)
(188, 128)
(188, 128)
(220, 128)
(182, 128)
(199, 128)
(240, 128)
(158, 128)
(267, 128)
(289, 128)
(184, 128)
(319, 128)
(212, 128)
(160, 128)
(240, 128)
(254, 128)
(192, 128)
(164, 128)
(188, 128)
(171, 128)
(162, 128)
(188, 128)
(184, 128)
(169, 128)
(164, 128)
(195, 128)
(235, 128)
(224, 128)
(619, 128)
(171, 128)
(597, 128)
(162, 128)
(495, 128)
(188, 128)
(627, 128)
(188, 128)
(171, 128)
(188, 128)
(165, 128)
(165, 128)
(179, 128)
(169, 128)
(272, 128)
(188, 128)
(389, 128)
(205, 128)
(160, 128)
(597, 128)
(188, 128)
(207, 128)
(162, 128)
(173, 128)
(205, 128)
(214, 128)
(634, 128)
(177, 128)
(559, 128)
(167, 128)
(336, 128)
(447, 128)
(180, 128)
(203, 128)
(179, 128)
(180, 128)
(332, 128)
(171, 128)

(299, 128)
(177, 128)
(214, 128)
(162, 128)
(160, 128)
(351, 128)
(276, 128)
(484, 128)
(188, 128)
(263, 128)
(160, 128)
(171, 128)
(409, 128)
(169, 128)
(203, 128)
(576, 128)
(261, 128)
(162, 128)
(263, 128)
(314, 128)
(188, 128)
(212, 128)
(162, 128)
(227, 128)
(184, 128)
(218, 128)
(257, 128)
(162, 128)
(285, 128)
(171, 128)
(177, 128)
(190, 128)
(383, 128)
(229, 128)
(353, 128)
(274, 128)
(175, 128)
(188, 128)
(216, 128)
(188, 128)
(252, 128)
(194, 128)
(188, 128)
(177, 128)
(188, 128)
(188, 128)
(582, 128)
(634, 128)
(173, 128)
(244, 128)
(190, 128)
(158, 128)
(164, 128)
(289, 128)
(229, 128)
(218, 128)
(540, 128)
(467, 128)
(372, 128)
(165, 128)
(469, 128)
(297, 128)
(265, 128)
(188, 128)
(158, 128)
(329, 128)
(349, 128)
(188, 128)
(582, 128)
(201, 128)
(259, 128)
(224, 128)
(165, 128)
(210, 128)
(225, 128)
(173, 128)
(158, 128)
(317, 128)
(179, 128)
(177, 128)
(248, 128)
(597, 128)
(344, 128)
(259, 128)
(387, 128)
(250, 128)
(473, 128)
(188, 128)
(615, 128)
(207, 128)
(188, 128)

...

(535, 128)
(604, 128)
(302, 128)
(313, 128)
(182, 128)
(164, 128)
(280, 128)
(389, 128)
(171, 128)
(313, 128)
(225, 128)
(501, 128)
(171, 128)
(225, 128)
(197, 128)
(175, 128)
(284, 128)
(165, 128)
(186, 128)
(158, 128)
(164, 128)
(334, 128)
(278, 128)
(608, 128)
(186, 128)
(355, 128)
(267, 128)
(612, 128)
(175, 128)
(269, 128)
(619, 128)
(293, 128)
(374, 128)
(257, 128)
(235, 128)
(190, 128)
(265, 128)
(300, 128)
(165, 128)
(167, 128)
(415, 128)
(203, 128)
(171, 128)
(194, 128)
(313, 128)
(177, 128)
(240, 128)
(179, 128)
(162, 128)
(179, 128)
(162, 128)
(203, 128)
(194, 128)
(212, 128)
(619, 128)
(302, 128)
(194, 128)
(229, 128)
(180, 128)
(175, 128)
(313, 128)
(231, 128)
(169, 128)
(195, 128)
(345, 128)
(186, 128)
(175, 128)
(165, 128)
(304, 128)
(304, 128)
(627, 128)
(313, 128)
(212, 128)
(370, 128)
(165, 128)
(295, 128)
(239, 128)
(313, 128)
(225, 128)
(313, 128)
(173, 128)
(257, 128)
(158, 128)
(261, 128)
(280, 128)
(360, 128)
(313, 128)
(165, 128)
(325, 128)
(210, 128)
(313, 128)

(177, 128)
(317, 128)
(634, 128)
(220, 128)
(212, 128)
(604, 128)
(173, 128)
(627, 128)
(265, 128)
(158, 128)
(182, 128)
(171, 128)
(600, 128)
(297, 128)
(169, 128)
(192, 128)
(359, 128)
(162, 128)
(194, 128)
(313, 128)
(179, 128)
(357, 128)
(257, 128)
(192, 128)
(199, 128)
(597, 128)
(160, 128)
(195, 128)
(233, 128)
(216, 128)
(239, 128)
(177, 128)
(557, 128)
(180, 128)
(225, 128)
(554, 128)
(464, 128)
(218, 128)
(529, 128)
(582, 128)
(224, 128)
(186, 128)
(255, 128)
(278, 128)
(216, 128)
(164, 128)
(167, 128)
(347, 128)
(634, 128)
(194, 128)
(593, 128)
(162, 128)
(195, 128)
(184, 128)
(220, 128)
(615, 128)
(199, 128)
(259, 128)
(488, 128)
(199, 128)
(287, 128)
(165, 128)
(203, 128)
(233, 128)
(623, 128)
(385, 128)
(169, 128)
(244, 128)
(184, 128)
(289, 128)
(280, 128)
(164, 128)
(167, 128)
(482, 128)
(297, 128)
(276, 128)
(313, 128)
(171, 128)
(313, 128)
(184, 128)
(490, 128)
(184, 128)
(313, 128)
(313, 128)
(173, 128)
(162, 128)
(195, 128)
(164, 128)
(608, 128)
(169, 128)
(313, 128)

(319, 128)
(539, 128)
(192, 128)
(237, 128)
(197, 128)
(259, 128)
(364, 128)
(634, 128)
(167, 128)
(171, 128)
(216, 128)
(216, 128)
(237, 128)
(299, 128)
(162, 128)
(205, 128)
(201, 128)
(179, 128)
(554, 128)
(282, 128)
(353, 128)
(214, 128)
(630, 128)
(276, 128)
(165, 128)
(190, 128)
(308, 128)
(194, 128)
(179, 128)
(462, 128)
(627, 128)
(205, 128)
(216, 128)
(205, 128)
(293, 128)
(400, 128)
(218, 128)
(235, 128)
(269, 128)
(182, 128)
(284, 128)
(608, 128)
(207, 128)
(201, 128)
(220, 128)
(239, 128)
(280, 128)
(313, 128)
(220, 128)
(182, 128)
(210, 128)
(233, 128)
(486, 128)
(173, 128)
(299, 128)
(347, 128)
(471, 128)
(167, 128)
(255, 128)
(216, 128)
(259, 128)
(182, 128)
(270, 128)
(186, 128)
(182, 128)
(222, 128)
(186, 128)
(165, 128)
(203, 128)
(177, 128)
(604, 128)
(207, 128)
(216, 128)
(225, 128)
(512, 128)
(225, 128)
(597, 128)
(197, 128)
(164, 128)
(182, 128)
(291, 128)
(162, 128)
(313, 128)
(197, 128)
(255, 128)
(269, 128)
(216, 128)
(608, 128)
(355, 128)
(282, 128)
(597, 128)

(313, 128)
(233, 128)
(229, 128)
(270, 128)
(203, 128)
(199, 128)
(454, 128)
(255, 128)
(240, 128)
(250, 128)
(229, 128)
(186, 128)
(164, 128)
(272, 128)
(162, 128)
(218, 128)
(313, 128)
(359, 128)
(224, 128)
(171, 128)
(254, 128)
(360, 128)
(162, 128)
(201, 128)
(182, 128)
(190, 128)
(627, 128)
(389, 128)
(173, 128)
(192, 128)
(186, 128)
(171, 128)


<Figure size 432x288 with 0 Axes>

In [ ]:
# выбираем 0,3 данных от всего trainig set для validation set
def val_make (source, dest):
    files = os.listdir(source)
    for f in files:
        if np.random.rand(1) < 0.3:
            shutil.move(source + '/'+ f, dest + '/'+ f)
    return 

In [ ]:
val_make(train_img_dir + "spoof", val_img_dir + "spoof") 

In [ ]:
val_make(train_img_dir + "human", val_img_dir + "human")

In [10]:
#создаем генераторы
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 20,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.15,
                                   zoom_range = 0.15,
                                   horizontal_flip = True,
                                   fill_mode="nearest")

val_datagen = ImageDataGenerator(rescale = 1./255)


In [12]:
#характиристики датасета
train_batch_size = 32
val_batch_size = 32
flow_from_directory = 224
img_height = 432
img_width = 288
#подготавливаем данные
train_gen = train_datagen.flow_from_directory(train_img_dir,
                                              batch_size = train_batch_size,
                                              class_mode = 'binary',
                                              target_size = (img_height, img_width),
                                              seed = seed_number)

val_gen = val_datagen.flow_from_directory(val_img_dir,
                                          batch_size = val_batch_size,
                                          class_mode = 'binary',
                                          target_size = (img_height, img_width),
                                          seed = seed_number)

Found 50000 images belonging to 2 classes.


In [13]:
#узнаем размерность
print(f'Train set batch shape\t: {next(train_gen)[0].shape}')


Train set batch shape	: (32, 432, 288, 3)


In [15]:
#строим модель
def build_model(input_shape):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((3, 3),  padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((3, 3), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((3, 3), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((3, 3), padding='same'))
    model.add(BatchNormalization())

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((3, 3), padding='same'))
    model.add(BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(Flatten())
    model.add(Dense(128, activation='relu',  activity_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dropout(rate=0.2))
    # output layer
    model.add(Dense(1, activation='sigmoid'))
    return model

In [16]:
#константы. кол-во эпох обучения и рэйт обучения
num_epochs = 15
learning_rate = 5e-5  

In [17]:
#создаем оптимизатор и компилируем модель
shape = [img_width, img_height, 3]
optimiser = tensorflow.keras.optimizers.Adam(learning_rate=learning_rate)
    
    
model = build_model(shape)
model.compile(optimizer=optimiser,
                  loss='binary_crossentropy',
                  metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 286, 430, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 96, 144, 32)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 144, 32)       128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 94, 142, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 48, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 48, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 46, 64)        3

In [18]:
#веса классов
counter = Counter(train_gen.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}   

print(class_weights)

{0: 3.84402247626429, 1: 1.0}


In [ ]:
#планировщик обучения
plateau_scheduler = ReduceLROnPlateau(factor=0.2, patience=3, verbose=1, 
                                      min_delta= 0.005, min_lr=5e-7)

In [ ]:
#это не нужно.  останавливает обучение, когда точность не растет в теч 3 эпох от макисмальной
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='acc', 
    min_delta=0.001,
    patience=3,
    mode='auto',
    verbose=1,
    baseline=None
)


In [14]:
train_length = len(train_gen.classes)
print(train_length)

50000


In [ ]:
#запускаем
history = model.fit(train_gen,
                    epochs = num_epochs,
                    steps_per_epoch = train_length // train_batch_size,
                    validation_data = val_gen,
                    validation_steps = 1,
                    callbacks = [plateau_scheduler],
                    class_weight=class_weights)

history_df = pd.DataFrame.from_dict(history.history)
history_df.to_csv(os.path.join(save_dir, "history_06_11.csv"), index=False)

In [15]:
#выводит графики для наглядности
train_accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(train_accuracy))
plt.figure(figsize=(12,4))

# точность
plt.subplot(1,2,1)
plt.plot(epochs, train_accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'], loc='lower right')

# потери
plt.subplot(1,2,2)
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'val'], loc='upper right')

plt.show()

NameError: name 'history' is not defined

In [ ]:
#переводит тестовый набор данных в спектограммы
classes = ['test']
for g in classes:
    pathlib.Path(f'img_data_test/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(input_dir + f'Testing_Data/{g}'):
        wav_file_name = input_dir + f'Testing_Data/{g}/{filename}'
        y, sr = librosa.load(wav_file_name, mono=True, duration=5)
        print(y.shape)
        plt.specgram(y, NFFT=n_fft, Fs=2, Fc=0, noverlap=128, cmap='plasma', sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data_test/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

In [ ]:
new_model = tf.keras.models.load_model('saved_model_sound') #загружаем модель

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255) #загружаем генератор

In [ ]:
#подготавилваем
test_gen = test_datagen.flow_from_directory(root,
                                              batch_size = 1,
                                              class_mode = None,
                                              classes=['img_data_test'],
                                              target_size = (img_width, img_height),
                                              seed = seed_number,
                                              shuffle=False)

In [ ]:
prediction = new_model.predict(test_gen,verbose=1,steps=len(test_gen)) #делаем предикт

In [ ]:
predicted_class = np.argmax(prediction,axis=-1)

In [ ]:
filenames=test_gen.filenames

In [ ]:
results=pd.DataFrame({"file":filenames,"pred":prediction[:,0]} )

In [ ]:
results.to_excel("output.xlsx")#сохраняем в формате excel 